In [1]:
from suffix_trees import STree
import suffix_tree
import pandas as pd

In [2]:
class STreeEx():
    def __init__(self, sTree):
        # 傳入原套件後綴樹
        self.sTree = sTree

    def lrs(self):
        # 最深非葉節點
        deepestNode = self._find_lrs(self.sTree.root)
        start = deepestNode.idx
        end = deepestNode.idx + deepestNode.depth
        return self.sTree.word[start:end]

    def _find_lrs(self, node):
        nodes = [self._find_lrs(n)
            for (n,_) in node.transition_links
            # 排除葉節點
            if n.transition_links != []]

        if nodes == []:
            return node
        
        deepestNode = max(nodes, key=lambda n: n.depth)
        return deepestNode

# Input File

In [3]:
input_file_path = "./input/TableA.txt"

In [4]:
recb_start = 0
recb_end = 0
pathid = []
parentid = []
tecid = []
cecid = []
encoding = []
content = []
col = []
with open(input_file_path) as file:
    line = file.readline()
    while(line):
        line = file.readline()
        tmp = line.split('\t')
        if tmp[0] == "RecB":
            tmp = tmp[1:]
            for index in range(len(tmp)):
                if tmp[index] == "SDR":
                    recb_start = index
                elif tmp[index] == "EDR":
                    recb_end = index
        elif tmp[0] == "PathId":
            pathid = tmp[1:]
        elif tmp[0] == "ParentId":
            parentid = tmp[1:]
        elif tmp[0] == "TECId":
            tecid = tmp[1:]
        elif tmp[0] == "CECId":
            cecid = tmp[1:]
        elif tmp[0] == "Encoding":
            encoding = tmp[1:]
        elif tmp[0] == "0":
            content = tmp[1:]
        elif tmp[0] == "Col Type":
            col = tmp[1:]
print(pathid[0] + "," + parentid[0] + "," + tecid[0] + "," + cecid[0] + "," + encoding[0] + "," + col[0])

0,"1:[1, 2]",0-2, ,1,MD


# Translate node in to ascii code for suffix tree

In [5]:
whole_string = ""
node_dict = {}
code_dict = {}
index_dict = {}
index = 0
for node in range(len(pathid)):
    code = pathid[node] + "," + parentid[node] + "," + tecid[node] + "," + cecid[node] + "," + encoding[node] + "," + col[node]
    if code not in node_dict.keys():
        node_dict[code] = index
        code_dict[code] = content[node]
        index_dict[code] = node
        whole_string += chr(index)
        index += 1
    else:
        whole_string += chr(node_dict[code])
inv_node_dict = {v: k for k, v in node_dict.items()}
print("Convert to ascii String:\n", whole_string)

Convert to ascii String:
  	
 !"#$%&'()* !"#$%&'()*+,-.-/-0-1-2-3-4-5-6-7-8-9-:-;-<-=->-?-@-A-B-C-D-E-F-G-H-IJKLMNOPQRSIJKTUVPQRWIJKTMNOPQRWIJKTXNOPQRSIJKTXNYPQRWIJKTXNOPQRWIJKLXNOPQRWIJKTXNOPQRWIJKTZ[PQRSIJKT\NOPQRSIJKT\NYPQRWIJK]^OPQRW_`abcbdbebfghijklmnopqrstuvwxyz{|}~


# Cut String from SRD to ERD

In [6]:
rec_string = whole_string[recb_start:recb_end+1]
print("Desire String:\n", rec_string)

Desire String:
 IJKLMNOPQRSIJKTUVPQRWIJKTMNOPQRWIJKTXNOPQRSIJKTXNYPQRWIJKTXNOPQRWIJKLXNOPQRWIJKTXNOPQRWIJKTZ[PQRSIJKT\NOPQRSIJKT\NYPQRWIJK]^OPQRW_`


# Longest repeat pattern

In [7]:
st = STree.STree(whole_string)
st = STreeEx(st)
lrs = str(st.lrs())
print("Length: ", len(lrs))
print(lrs)
for i in lrs:
    print(inv_node_dict[ord(i)], "\tContent: ", code_dict[inv_node_dict[ord(i)]])

Length:  26
 !"#$%&'()*
6,5:[1],10-6,6-30,0,MT 	Content:  Lowest price :: 29 :: 
6,5:[1],11-8,6-19,0,MT 	Content:  # :: 30 :: 
6,5:[1],12-6,6-31,0,MT 	Content:  All price :: 31 :: 
6,5:[1],13-8,6-19,0,MT 	Content:  # :: 32 :: 
6,5:[1],12-9,6-32,0,MT 	Content:  Free :: 33 :: 
6,5:[1],13-8,6-19,0,MT 	Content:  # :: 32 :: 
6,5:[1],12-15,6-33,0,MT 	Content:  TWD 1-1000 :: 35 :: 
6,5:[1],13-8,6-19,0,MT 	Content:  # :: 32 :: 
6,5:[1],12-15,6-34,0,MT 	Content:  TWD 1000-2000 :: 37 :: 
6,5:[1],13-8,6-19,0,MT 	Content:  # :: 32 :: 
6,5:[1],12-15,6-35,0,MT 	Content:  TWD 2000-3000 :: 39 :: 
6,5:[1],13-8,6-19,0,MT 	Content:  # :: 32 :: 
6,"1:[1, 2]",12-16,6-36,0,MT 	Content:  Upon TWD 3000 :: 41 :: 
6,5:[1],13-8,6-19,0,MT 	Content:  # :: 32 :: 
6,5:[1],10-9,6-37,0,MT 	Content:  Time :: 43 :: 
6,5:[1],11-8,6-19,0,MT 	Content:  # :: 30 :: 
6,5:[1],12-6,6-38,0,MT 	Content:  All time :: 45 :: 
6,5:[1],13-8,6-19,0,MT 	Content:  # :: 32 :: 
6,5:[1],12-9,6-39,0,MT 	Content:  Today :: 47 :

In [8]:
st2 = STree.STree(rec_string)
st2 = STreeEx(st2)
rec_lrs = str(st2.lrs())
print("Length: ", len(rec_lrs))
print(rec_lrs)
for i in rec_lrs:
    print(inv_node_dict[ord(i)], "\tContent: ", code_dict[inv_node_dict[ord(i)]])

Length:  18
PQRWIJKTXNOPQRWIJK
6,5:[1],24-8,6-79,0,MT 	Content:  ( :: 145 :: 
6,5:[1],24-9, ,1,MD 	Content:  Tue :: 146 :: 
6,5:[1],24-8,6-81,0,MT 	Content:  ) :: 147 :: 
6,5:[1],23-9, ,1,MD 	Content:  Available :: 159 :: 
6,3:[4],17-4, ,1,MD 	Content:  https://t.kfs.io/upload_images/105724/_____2019-11-26___4.50.59_medium.png :: 138 :: 
6,3:[4],18-4, ,1,MD 	Content:  https://innovation.kktix.cc/events/vote-the-works-2020 :: 139 :: 
6,5:[1],19-9, ,1,MD 	Content:  Gathering :: 140 :: 
7,5:[1],20-5, ,2,MD 	Content:  【職前訓練】跨平台網站製作-介面規劃與RWD前端程式開發班 :: 152 :: 
7,5:[1],21-5, ,2,MD 	Content:  Longboard初階課 課程時間：2 小時 課程費用：$700/人 初階真的很重要！因為你將學習正確的滑行動作及重心調整，並可享受到最真的滑行樂趣。 課程內容：方向性滑行、單雙腳橫向平衡訓練、腳煞、 :: 175 :: 
7,"1:[1, 2]",22-18, ,1,OD 	Content:  113 Attendees :: 143 :: 
6,5:[1],23-0, ,3,MD 	Content:  2020/1/7 :: 144 :: 
6,5:[1],24-8,6-79,0,MT 	Content:  ( :: 145 :: 
6,5:[1],24-9, ,1,MD 	Content:  Tue :: 146 :: 
6,5:[1],24-8,6-81,0,MT 	Content:  ) :: 147 :: 
6,5:[1],23-9, ,1,MD 	Content:  Available ::

# Show maximal repeats

Ignore maximal repeats if length equal to 1.

In [9]:
tree = suffix_tree.Tree({'A': whole_string})
print("Total Maximal repeats: ", len(tree.maximal_repeats()), '\n')
group = {}
for C, path in tree.maximal_repeats():
    if len(path) == 1: continue
    print("Count: ", C, "Length: ", len(path), "Pattern: {", path, "}")
    print("Original code:")
    for i in path:
        print(inv_node_dict[ord(i)], "\tContent: ", code_dict[inv_node_dict[ord(i)]])
    if path[0] not in group.keys():
        group[path[0]] = []
        group[path[0]].append(path)
    else:
        group[path[0]].append(path)
    print("\n", "="*50, "\n")

Total Maximal repeats:  32 

Count:  1 Length:  26 Pattern: {        !  "  #  $  %  &  '  (  )  *  }
Original code:
6,5:[1],10-6,6-30,0,MT 	Content:  Lowest price :: 29 :: 
6,5:[1],11-8,6-19,0,MT 	Content:  # :: 30 :: 
6,5:[1],12-6,6-31,0,MT 	Content:  All price :: 31 :: 
6,5:[1],13-8,6-19,0,MT 	Content:  # :: 32 :: 
6,5:[1],12-9,6-32,0,MT 	Content:  Free :: 33 :: 
6,5:[1],13-8,6-19,0,MT 	Content:  # :: 32 :: 
6,5:[1],12-15,6-33,0,MT 	Content:  TWD 1-1000 :: 35 :: 
6,5:[1],13-8,6-19,0,MT 	Content:  # :: 32 :: 
6,5:[1],12-15,6-34,0,MT 	Content:  TWD 1000-2000 :: 37 :: 
6,5:[1],13-8,6-19,0,MT 	Content:  # :: 32 :: 
6,5:[1],12-15,6-35,0,MT 	Content:  TWD 2000-3000 :: 39 :: 
6,5:[1],13-8,6-19,0,MT 	Content:  # :: 32 :: 
6,"1:[1, 2]",12-16,6-36,0,MT 	Content:  Upon TWD 3000 :: 41 :: 
6,5:[1],13-8,6-19,0,MT 	Content:  # :: 32 :: 
6,5:[1],10-9,6-37,0,MT 	Content:  Time :: 43 :: 
6,5:[1],11-8,6-19,0,MT 	Content:  # :: 30 :: 
6,5:[1],12-6,6-38,0,MT 	Content:  All time :: 45 :: 
6

# Group by first element

In [10]:
group_num = 0
group_2 = {}
for team in group:
    print("Group ", group_num, " start element: ", code_dict[inv_node_dict[ord(team)]], " start index: ", index_dict[inv_node_dict[ord(team)]])
    print("Count: ", len(group[team]), "\n\n", "="*100, "\n")
    if len(group_2) == 0:
        group_2[index_dict[inv_node_dict[ord(team)]]] = []
        for path in group[team]:
            group_2[index_dict[inv_node_dict[ord(team)]]].append(path)
    else:
        check = 0
        for group_2_idx in group_2.keys():
            if index_dict[inv_node_dict[ord(team)]] - group_2_idx <= 36:
                for path in group[team]:
                    group_2[group_2_idx].append(path)
                check = 1
                break
        if not check:
            group_2[index_dict[inv_node_dict[ord(team)]]] = []
            for path in group[team]:
                group_2[index_dict[inv_node_dict[ord(team)]]].append(path)
    group_num += 1

Group  0  start element:  Lowest price :: 29 ::   start index:  29
Count:  1 


Group  1  start element:  https://t.kfs.io/upload_images/105724/_____2019-11-26___4.50.59_medium.png :: 138 ::   start index:  138
Count:  4 


Group  2  start element:  經過了三個禮拜的激戰，各方好手此時此刻正齊聚一堂，拿出最終實力一較高下 ; 除了實力的比拼以外，人氣指數也是選手們獲得獎勵的指標之一！話不多說，趕緊為你心中所屬的隊伍投下神聖的一票吧！ :: 142 ::   start index:  142
Count:  1 


Group  3  start element:  113 Attendees :: 143 ::   start index:  143
Count:  6 


Group  4  start element:  2020/1/7 :: 144 ::   start index:  144
Count:  2 


Group  5  start element:  ( :: 145 ::   start index:  145
Count:  8 


Group  6  start element:  Longboard初階課 課程時間：2 小時 課程費用：$700/人 初階真的很重要！因為你將學習正確的滑行動作及重心調整，並可享受到最真的滑行樂趣。 課程內容：方向性滑行、單雙腳橫向平衡訓練、腳煞、 :: 175 ::   start index:  174
Count:  4 




In [11]:
for team in group_2:
    print("="*100, "\nTeam:", team, "\n")
    for path in group_2[team]:
        print(path)
sequences = [str(path) for path in group_2[138]]

Team: 29 

       !  "  #  $  %  &  '  (  )  * 
Team: 138 

I J K
I J K L
I J K T
I J K T X N
M N O P Q R
N O P Q R
N O P Q R S I J K T
N O P Q R W I J K
N O P Q R W I J K T
N O P Q R W I J K T X N O P Q R
N Y P Q R W I J K
O P Q R
O P Q R W
P Q R
P Q R S I J K T
P Q R S I J K T \ N
P Q R W
P Q R W I J K
P Q R W I J K T
P Q R W I J K T X N O P Q R
P Q R W I J K T X N O P Q R W I J K
X N
X N O P Q R
X N O P Q R W I J K
X N O P Q R W I J K T


In [12]:
import cstar
scores = [1, -1, -4]
msa = cstar.CenterStar(scores, sequences).msa()

"""with open(args.outputfile, 'w') as out:
    out.writelines(map(lambda x: x + '\n', msa))"""
for item in list(map(lambda x: x + '\n', msa)):
    print(item)

--------------I J K T--------------------

--------------I J K----------------------

--------------I J K L--------------------

--------------I J K T X N----------------

M N ----------O P Q R--------------------

N ------------O P Q R--------------------

N O P Q R S --I J K T--------------------

N O P Q R ----W I J K--------------------

N O P Q R W --I J K T--------------------

N O P Q R W --I J K T X N O P Q R--------

N Y P Q R ----W I J K--------------------

--------------O P Q R--------------------

O ------------P Q R W--------------------

----------------P Q R--------------------

P Q R S ------I J K T--------------------

P Q R S ------I J K T \ N----------------

--------------P Q R W--------------------

P Q R --------W I J K--------------------

P Q R W ------I J K T--------------------

P Q R W ------I J K T X N O P Q R--------

P Q R W ------I J K T X N O P Q R W I J K

------------------X N--------------------

X N ----------O P Q R--------------------

X N O P Q R